### RNN Model Implementation

#### 1. torchtext 설치 및 hyperparameter 설정

In [ ]:
# %pip install torchtext==0.6

In [1]:
import os
import torch
import torch.nn as nn
from torchtext import data, datasets
import random

In [2]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
# [batch_size(:하나의 batch 내에 들어있는 문장 개수),  문장 내 단어 개수]
# [enforce fail at cpuallocator.cpp:68] . defaultCPUallocator: can't allocate memory: you tried to allocate 8247153600 bytes. error code 12 (cannot allocate memory)
# batch_size = 256 으로 설정했을 때 발생.
# CPU가 감당 못하는 너무 큰 메모리를 할당하려고 할 때 발생하는 에러 -> 실제로 학습시간이 1시간을 넘어갔다.
# batch_size = 64 로 변경
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# CUDA(Compute Unified Device Architecture)
# GPU의 가상 명령어셋을 사용할 수 있도록 만들어주는 소프트웨어 레이어로, 
# NVIDIA가 만든 CUDA 코어가 장착된 GPU에서 작동 Many-Core dependent 연산
# 많은 양의 연산을 동시에 처리

#### 2. torchtext.data.Field 에 따른 데이터 불러오기

In [ ]:
# Preprocessing

TEXT = data.Field(sequential=True, use_vocab=True, batch_first=True,lower=True)
LABEL = data.Field(sequential=false, use_vocab=True, batch_first=True)

# Distribute the Train / Test data in 1:1
train_dataset, test_dataset = datasets.IMDB.splits(TEXT, LABEL)

print(len(train_dataset), len(test_dataset))
print('Print the elements of train_set : ', train_dataset.fields)


In [ ]:
# vars(train_dataset[0])

#### 3. Generate Vocabulary Dictionary and set additional parameter

In [ ]:
TEXT.build_vocab(train_dataset, min_freq=5)
LABEL.build_vocab(train_dataset)
# label 에 해당하는 값이 정수가 아니라 'pos', 'neg' 와 같은 string 이기 때문에
# 이를 단어 사전 내의 정수 index 로 치환

In [ ]:
# 모델 선언에서 사용할 parameter
vocab_size = len(TEXT.vocab)
n_classes = 2
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

In [ ]:
print(TEXT.vocab.stoi)

In [ ]:
print(LABEL.vocab.stoi)

In [ ]:
# 훈련 / 검증 데이터 8:2 분리, label 비율 유지
train_set, validation_set = train_dataset.split(split_ratio=0.8, stratified=True, strata_field='label')

Transmit to Data Loader

In [4]:
from torchtext.data import Iterator

In [ ]:
train_iter, val_iter, test_iter = Iterator.splits((train_set, validation_set, test_dataset), 
                                                  batch_size=BATCH_SIZE, shuffle=True)

print('Number of mini-batch for train data : {}'.format(len(train_iter)))
print('Number of mini-batch for test data : {}'.format(len(test_iter)))
print('Number of mini-batch for validation data : {}'.format(len(val_iter)))

In [ ]:
batch = next(iter(train_iter))
print(batch.text.shape)
print()
print(batch.text)

In [ ]:
batch = next(iter(train_iter))
batch.text.shape() # MAX_LENGTH 에 따른 padding 처리가 안됨
# batch 안의 가장 긴 sentence의 길이가 MAX_LENGTH가 되어 padding

In [ ]:
y = batch.label
print(y)
y.sub_(1)
print(y)

In [ ]:
batch.text

In [ ]:
batch.text.data

In [ ]:
train_iter, val_iter, test_iter = Iterator.splits((train_set, validation_set, test_dataset), 
                                                  batch_size=BATCH_SIZE, shuffle=True)
# 앞서 next 했던 값을 다시 복원

#### 4. Model Implementation

<img src="./rnn_img/image.png"/>

In [ ]:
class GRU(nn.Module):
    def __init__(self,n_layers,hidden_dim,n_vocab,embed_dim,n_class,dropout_p=0.2):
        super(GRU,self).__init__()
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        self.embed=nn.Embedding(n_vocab,embed_dim)
        self.dropout=nn.Dropout(dropout_p)
        self.gru=nn.GRU(embed_dim,self.hidden_dim,num_layers=self.n_layers,batch_first=True)
        self.out=nn.Linear(self.hidden_dim,n_class)
        
    def forward(self,x):
        x=self.embed(x) # Embedding
        x,_=self.gru(x) # GRU
        last_h=x[:,-1,:] # extract last hidden state vector per sentence
        self.dropout(last_h) # Dropout
        logit=self.out(last_h) # Linear
        
        return logit

#### 5. Model Compling

In [ ]:
model = GRU(1, 64, vocab_size, 128, n_classes, 0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(reduction='sum').to(device)
# 하나의 batch 에 대한 loss 값을 모두 더한 것을 그 batch 의 total_loss로 설정

In [ ]:
def train(model,optimizer,criterion,train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(device), batch.label.to(device)
        
        # label의 값들 1,2 에서 1을 빼서 0,1로 변환
        # label_vocab : {'<unk>':0, 'neg':1, 'pos':2}
        # y 와 y.data는 동일 -> y.data.sub_(1) == y.sub_(1)
        y.data.sub_(1)
        
        logit = model(x)
        loss = criterion(logit, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#### 6. Functionalization for train and validation

In [ ]:
def evaluate(model,val_iter,criterion):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(device), batch.label.to(device)
        y.data.sub_(1)
        logit = model(x)
        loss = criterion(logit, y)
        total_loss += loss
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = corrects / size
    return avg_loss, avg_accuracy

<img src="./rnn_img/image (1).png"/>

In [ ]:
a = torch.Tensor([[1,0],[1,0],[0,1],[1,0]])
a.max(1)

In [ ]:
a.max(1)[1]

In [ ]:
y = torch.tensor([0,0,0,1])
y

In [ ]:
a.max(1)[1] == y

In [ ]:
(a.max(1)[1] == y).sum()

<img src="./rnn_img/image (2).png"/>

#### 7. Training

In [ ]:
best_val_loss = None # 최고 성능을 낸 검증 loss 값을 저장

for e in range(1, EPOCHS+1):
    train(model,optimizer,criterion,train_iter)
    val_loss, val_accuracy = evaluate(model,val_iter,criterion)
    
    print("Epoch : {} | Val_loss : {:5.2f}, Val_accuracy : {:5.2f}".format(e,val_loss,val_accuracy))
    
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(),'./snapshot/txtclassification.pt') # 가장 성능 좋은 가중치 저장
        best_val_loss = val_loss

#### 8. Evaluate Performance

In [ ]:
model.load_state_dict(torch.load('./snapshot/txtclassificaiton.pt'))
test_loss,test_acc=evaluate(model,test_iter,criterion)
print('테스트 오차: {:5.2f} | 테스트 정확도: {:5.2f}'.format(test_loss, test_acc))

<img src="./rnn_img/Untitled.png"/>
<img src="./rnn_img/Untitled_(1).png"/>
<img src="./rnn_img/Untitled_(2).png"/>
<img src="./rnn_img/Untitled_(3).png"/>
<img src="./rnn_img/Untitled_(4).png"/>